In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""

CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_logistic.a_daily_scct_sku_active_daily
AS
WITH ACT_SKU AS (
  SELECT
   A.calday,
   A.store_id,
   A.product_id,
   business_unit,
   mien AS region
  FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_active_his A
  INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store USING (store_id)
  INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product USING (product_id)
  WHERE A.calday BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),15) AND DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
    AND business_unit IN ('1500','2000')  
    AND mch2_id IN ('102','201','202','203')
),

ALL_STORE_SKU AS (
  SELECT A.*,
    B.internal_report_mdq AS mdq,
    CASE
      WHEN D.dc_id IS NOT NULL THEN 'DC'
      WHEN B.product_id IS NOT NULL THEN 'DSD'
      ELSE 'NA'
    END AS replenishment_mode,
    COALESCE(
      B.assortment_type,
      CASE
        WHEN A.calday < DATE('2024-04-02') THEN ASM_1.core_range
        WHEN A.calday >= DATE('2024-04-02') AND ST.business_unit = '1500' THEN ASM_3.core_range
        WHEN A.calday >= DATE('2024-04-02') AND ST.business_unit = '2000' THEN ASM_2.core_range
        ELSE 'NA'
      END,
      'NA'
    ) AS assortment_type,
    B.mdq_age,
    D.dc_id,
    is_problem_article
  FROM ACT_SKU A
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store ST ON A.store_id = ST.store_id
  LEFT JOIN (
    SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_daily_hist 
    WHERE calday BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),15) AND DATE('2024-08-08')
    UNION ALL
    SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_mdq_ambient_biweekly_hist
    WHERE calday >= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),15) AND calday < DATE('2024-08-08')
  ) B ON A.calday = B.calday AND A.store_id = B.store_id AND A.product_id = B.product_id
  LEFT JOIN (
    SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_mdq_store_dc_sku_daily_his
    WHERE calday >= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),15)
    QUALIFY ROW_NUMBER() OVER(PARTITION BY store_id, product_id, calday ORDER BY dc_id) = 1
  ) D ON A.store_id = D.store_id AND A.product_id = D.product_id AND A.calday = D.calday
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_his ASM_1
    ON A.calday = ASM_1.calday AND A.product_id = ASM_1.product_id AND ST.business_unit = ASM_1.business_unit AND ST.city = ASM_1.city
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmp_his ASM_2
    ON A.calday = ASM_2.calday AND A.product_id = ASM_2.product_id AND ST.business_unit = ASM_2.business_unit AND ST.city = ASM_2.city
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_core_range_assortment_wmt_his ASM_3
    ON A.calday = ASM_3.calday AND A.product_id = ASM_3.product_id AND A.store_id = ASM_3.store_id
),

QTY_DAILY AS (
  SELECT A.calday, A.store_id, A.product_id,
    SUM(COALESCE(B.revenue.base_sale_qty, 0)) AS sale_qty
  FROM ALL_STORE_SKU A
  LEFT JOIN (
    SELECT * FROM {catalog_name}.udp_wcm_gold_vcm_dmt.a_store_sku_daily
    WHERE calday >= DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),121)
  ) B ON A.calday = B.calday AND A.store_id = B.store_id AND A.product_id = B.product_id
  GROUP BY 1, 2, 3
),

QTYPOS_30D AS (
  SELECT *,
    SUM(sale_qty) OVER (PARTITION BY store_id, product_id ORDER BY calday ASC ROWS BETWEEN 29 PRECEDING AND CURRENT ROW) AS sale_qty_30d
  FROM QTY_DAILY
),

SUMM AS (
  SELECT A.calday, A.dc_id, A.store_id, A.product_id, A.mdq,
    A.replenishment_mode, A.assortment_type, 'YES' AS is_active,
    CASE WHEN mdq > 0 THEN 'YES' WHEN mdq = 0 THEN 'MDQ=0' ELSE 'MDQ IS NULL' END AS having_mdq,
    CASE WHEN replenishment_mode = 'NA' THEN 'NO' ELSE 'YES' END AS in_snapshot,
    CASE WHEN C.sku IS NULL THEN 'NO' ELSE 'YES' END AS is_problem,
    CASE WHEN D.store_id IS NOT NULL THEN 'YES' ELSE 'NO' END AS is_block,
    COALESCE(sale_qty_30d, 0)/30 AS ma30_pos
  FROM ALL_STORE_SKU A
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store ST USING (store_id)
  LEFT JOIN QTYPOS_30D B USING (calday, store_id, product_id)
  LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_db_inventory.d_problem_article_his C
    ON A.business_unit = C.business_unit AND A.product_id = C.sku AND ST.region_vn = C.area AND A.calday = C.calday
  LEFT JOIN (
    SELECT calday, store_id, product_id FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_cfg_winare_block_by_store_sku_current_his
    WHERE calday BETWEEN DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),15) AND DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),1)
      AND reason LIKE 'High stock%'
    UNION ALL
    SELECT DATE_SUB(DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS),1) AS calday, store_id, product_id
    FROM {catalog_name}.udp_wcm_silver_dataportal.d_cfg_winare_block_by_store_sku
    WHERE reason LIKE 'High stock%' AND to >= DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS)
  ) D ON A.calday = D.calday + 1 AND A.product_id = D.product_id AND A.store_id = D.store_id
)

SELECT * FROM SUMM;
""")